# Reshaping and tidy data

Documentation sources:

* [Hadley Wickham. Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf)
* https://tomaugspurger.github.io/modern-5-tidy
* http://matthewrocklin.com/blog/work/2015/06/18/Categoricals
* https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html


## I. Philosophy behind tidy data

* Data to be processed is rarely in the correct format for the analysis.
* One must reshape the data before it is in the convenient form for the analysis.

* A successful data cleaning pipeline is a compromise between two conflicting principles:

  * Trust no one!
  * Get things done!
  
  
* Normally, one starts with exploratory data analysis to find:
  
  * invariants to check
  * artefacts and input errors to correct
  * normalise information in the fields 
  * remove duplicated information
  * resolve inconsistencies between different fields


* At the end of this process, you should have tidy data where
  
  * each observed variable is a column of a data frame
  * each observation form a row of a data frame
  * each type of observational unit forms a data frame
  
  
* To illustrate the last condition, consider observational units of a supermarket data:
  
  * information about the buyer
  * time and payment type of a particular transaction
  * individual items in the transaction, including deductions

* If we are interested in who buys beer and when, then age, gender and item type must be in the data frame.
* If we are interested in how much a demographic segment buys on average and when, then only age, gender, time and sum must be kept.
* If we are doing more complex behavioral analysis, then we need two data frames:
   * one for describing persons
   * the other for describing purchases of individuals


In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import read_csv
from pandas import melt
from pandas import concat

## I. How to melt wide tables

* It is quite common that some column headers represent values, not variable names.
* For instance, some columns indicate dates or subpopulations.
* To tidy up this data, the table must be melted into long format.
* This can be achieved with `melt` function :
  * `id_vars`    – columns to be preserved as they are
  * `value_vars` – columns to be converted to value (usually not needed)
  * `var_name`   – name of the new variable
  * `value_name` – name of the observed value
  * `col_level`  – which multi-index levels to use 
* If the column index is a multi-index then we can specify which levels to keep but this can lead to information loss.

In [2]:
# Melt a wide table with a single header row
display(read_csv('estonian_population_1.csv', header = None).head())
df = read_csv('estonian_population_1.csv')
display(df.head())

# Select all years for melting
display(melt(df, id_vars=['Age', 'Gender'], var_name = 'Year', value_name = 'Count').tail())

# Select only years 1950 - 1951 for melting 
display(melt(df, id_vars=['Age', 'Gender'], var_name = 'Year', value_vars = ['1950', '1951'],  value_name = 'Count').tail())

# Melt a table with many header rows 
display(read_csv('estonian_population_2.csv', header = None).head())

# As the Age column is wrongly named we push it into the index 
df = read_csv('estonian_population_2.csv', header = [0,1], index_col = 0)
display(df.head())
display(melt(df, value_name = 'Count').reset_index().rename(columns = {'index': 'Age'}).head())

# It is possible to use only Gender and Year levels but this causes information loss
display(melt(df, col_level = 'Gender', value_name = 'Count').head())
display(melt(df, col_level = 'Year', value_name = 'Count').head())

0       1      2      3      4      5      6      7      8      9   ...  \
0  Age  Gender   1950   1951   1952   1953   1954   1955   1956   1957  ...   
1    0     All  19310  18841  19356  19563  19239  20441  20038  19242  ...   
2    0  Female   9303   9042   9298   9458   9260   9835   9784   9326  ...   
3    0    Male  10007   9799  10058  10105   9979  10606  10254   9916  ...   
4    1     All  18162  18251  17743  18368  19069  18473  19534  19631  ...   

      42     43     44     45     46     47     48     49     50     51  
0   1990   1991   1992   1993   1994   1995   1996   1997   1998   1999  
1  24006  22041  18956  17571  14794  13760  13192  12981  12326  11911  
2  11637  10689   9224   8545   7116   6756   6430   6320   5931   5761  
3  12369  11352   9732   9026   7678   7004   6762   6661   6395   6150  
4  24542  23858  21841  18506  17332  14589  13602  13053  12839  12178  

[5 rows x 52 columns]

Age  Gender   1950   1951   1952   1953   1954   1955   1956   1957  ...  \
0    0     All  19310  18841  19356  19563  19239  20441  20038  19242  ...   
1    0  Female   9303   9042   9298   9458   9260   9835   9784   9326  ...   
2    0    Male  10007   9799  10058  10105   9979  10606  10254   9916  ...   
3    1     All  18162  18251  17743  18368  19069  18473  19534  19631  ...   
4    1  Female   8856   8889   8609   8910   9291   8939   9473   9644  ...   

    1990   1991   1992   1993   1994   1995   1996   1997   1998   1999  
0  24006  22041  18956  17571  14794  13760  13192  12981  12326  11911  
1  11637  10689   9224   8545   7116   6756   6430   6320   5931   5761  
2  12369  11352   9732   9026   7678   7004   6762   6661   6395   6150  
3  24542  23858  21841  18506  17332  14589  13602  13053  12839  12178  
4  12179  11577  10587   8995   8441   7019   6686   6372   6253   5855  

[5 rows x 52 columns]

Age  Gender  Year  Count
12745   83  Female  1999   2817
12746   83    Male  1999    950
12747   84     All  1999   3924
12748   84  Female  1999   3002
12749   84    Male  1999    922

Age  Gender  Year  Count
505   83  Female  1951   1099
506   83    Male  1951    466
507   84     All  1951   1484
508   84  Female  1951   1072
509   84    Male  1951    412

0      1       2      3      4       5     6      7       8      9    \
0    Year   1950    1950   1950   1951    1951  1951   1952    1952   1952   
1  Gender    All  Female   Male    All  Female  Male    All  Female   Male   
2       0  19310    9303  10007  18841    9042  9799  19356    9298  10058   
3       1  18162    8856   9306  18251    8889  9362  17743    8609   9134   
4       2  16928    8178   8750  18209    8863  9346  18295    8882   9413   

   ...   141    142     143   144    145     146   147    148     149   150  
0  ...  1996   1997    1997  1997   1998    1998  1998   1999    1999  1999  
1  ...  Male    All  Female  Male    All  Female  Male    All  Female  Male  
2  ...  6762  12981    6320  6661  12326    5931  6395  11911    5761  6150  
3  ...  6916  13053    6372  6681  12839    6253  6586  12178    5855  6323  
4  ...  7505  13487    6628  6859  12946    6326  6620  12746    6210  6536  

[5 rows x 151 columns]

Year     1950                 1951                1952                 1953  \
Gender    All Female   Male    All Female  Male    All Female   Male    All   
0       19310   9303  10007  18841   9042  9799  19356   9298  10058  19563   
1       18162   8856   9306  18251   8889  9362  17743   8609   9134  18368   
2       16928   8178   8750  18209   8863  9346  18295   8882   9413  17744   
3       16540   8117   8423  17059   8274  8785  18271   8919   9352  18355   
4       13620   6680   6940  16735   8201  8534  17211   8343   8868  18368   

Year    ...  1996   1997                1998                1999               
Gender  ...  Male    All Female  Male    All Female  Male    All Female  Male  
0       ...  6762  12981   6320  6661  12326   5931  6395  11911   5761  6150  
1       ...  6916  13053   6372  6681  12839   6253  6586  12178   5855  6323  
2       ...  7505  13487   6628  6859  12946   6326  6620  12746   6210  6536  
3       ...  8705  14375   6926  7449  13414   6598  6816  12868   6292  6576  
4       ...  9101  16863   8225  8638  14317   6904  7413  13329   6562  6767  

[5 rows x 150 columns]

Age  Year Gender  Count
0    0  1950    All  19310
1    1  1950    All  18162
2    2  1950    All  16928
3    3  1950    All  16540
4    4  1950    All  13620

Gender  Count
0    All  19310
1    All  18162
2    All  16928
3    All  16540
4    All  13620

Year  Count
0  1950  19310
1  1950  18162
2  1950  16928
3  1950  16540
4  1950  13620

## II. What to do when a column encodes several variables

* Sometimes a value in a table encodes several variables.
* In the simplest case, this information is encoded into the value:
  * age groups `M11-15`, `F11-15`, `M16-20`,...
  * codes of time periods `2012Q1`, `2012-01-01 10:30`
  * international phone numbers `<country> <number>`
* Then string manipulation functions become handy.
* In complex cases, the information is external or must be manually added:
  * item codes in supermarkets can be divided into food groups
  * billed sum in different currencies but the currency information is lost
* The last example is quite hard to handle: 
  * If there is a pricelist then currency can be deduced.
  * If there is a persons IP address then currency can be deduced form the approximate geographical location.
  * The latter is heuristic and requires a lot of manual work.

In [3]:
df = read_csv('pricelist.csv')
df.head(10)

Motherboard CPU socket Form factor  RAM slots Max RAM  \
0                 Asus Prime Z370-A    LGA1151         ATX          4   64 GB   
1                  MSI B350 PC MATE        AM4         ATX          4   64 GB   
2                    MSI Z370-A PRO    LGA1151         ATX          4   64 GB   
3               Gigabyte B450M DS3H        AM4   Micro ATX          4   64 GB   
4                 MSI B450 TOMAHAWK        AM4         ATX          4   64 GB   
5      Asus ROG STRIX Z390-E GAMING    LGA1151         ATX          4   64 GB   
6          Asus STRIX B350-F GAMING        AM4         ATX          4   64 GB   
7      Asus ROG STRIX B450-F GAMING        AM4         ATX          4   64 GB   
8                  ASRock B450M-HDV        AM4   Micro ATX          2   32 GB   
9  Gigabyte X470 AORUS ULTRA GAMING        AM4         ATX          4   64 GB   

     Price  
0  $173.98  
1   $85.49  
2  $109.99  
3   $76.88  
4  $110.33  
5  $235.99  
6  $115.53  
7  $129.00  
8   $79.89  
9  $119.99

### String manipulation

* Pandas provides a powerful toolset for string manipulation.
* All of them go through `str` attribute that provides vectorized string operations:
  * `str[a:b]` – shorcut for substring selection
  * `str.strip` – remove leading and trailing whitespaces or other characters
  * `str.split` – split string based on separator pattern 
  * `str.extract` – extracts the first substring corresponding to a regex pattern
  * `str.extractall` – extracts all substrings corresponding to a regex pattern
  * `str.findall` – find all occurences of a regular expression
  * `str.contains` – detects if a string contains a regular expression
  * `str.matches` – detects if regular expression matches string 
  * `str.replace` – replaces a regex pattern with new text
  
* The complete list of operations with further details can be found:
  * https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html
* Most advanced operations require knowledge of regex patterns:
  * https://docs.python.org/3/howto/regex.html
  * https://www.tutorialspoint.com/python/python_reg_expressions.htm
  * https://www.datacamp.com/community/tutorials/python-regular-expression-tutorial
* Key things to remember about regexes:
  * raw strings `r'<text>'` make regex writing and reading tracktable
  * there are wildcards for important patterns: whitespaces, numbers, dates, etc
  * syntax `(?P<name>...)` allows to name capture groups
  * syntax `(?:...)` allows to drop a capture group
  * symbols `*`, `+`, `?` allow to specify optional parts of the pattern
  * capture groups `\<number>` and `\g<name>` can be referenced in a replacement pattern 

In [4]:
# Simple substring selection to drop dollar sign
display(df.assign(Price = df['Price'].str[1:].astype(float)).head())

# Using right strip to drop GB at the end. Bonus how to assign to column with whitespace
display(df.assign(**{'Max RAM': df['Max RAM'].str.rstrip('GB').astype(float)}).head())

# Split the column and join it with the original data frame
display(df.join(df['Max RAM'].str.split(expand = True)
                .rename(columns = {0: 'Max RAM size', 1:'Max RAM Unit'}))
        .drop(columns=['Max RAM']).head())

Motherboard CPU socket Form factor  RAM slots Max RAM   Price
0    Asus Prime Z370-A    LGA1151         ATX          4   64 GB  173.98
1     MSI B350 PC MATE        AM4         ATX          4   64 GB   85.49
2       MSI Z370-A PRO    LGA1151         ATX          4   64 GB  109.99
3  Gigabyte B450M DS3H        AM4   Micro ATX          4   64 GB   76.88
4    MSI B450 TOMAHAWK        AM4         ATX          4   64 GB  110.33

Motherboard CPU socket Form factor  RAM slots  Max RAM    Price
0    Asus Prime Z370-A    LGA1151         ATX          4     64.0  $173.98
1     MSI B350 PC MATE        AM4         ATX          4     64.0   $85.49
2       MSI Z370-A PRO    LGA1151         ATX          4     64.0  $109.99
3  Gigabyte B450M DS3H        AM4   Micro ATX          4     64.0   $76.88
4    MSI B450 TOMAHAWK        AM4         ATX          4     64.0  $110.33

Motherboard CPU socket Form factor  RAM slots    Price  \
0    Asus Prime Z370-A    LGA1151         ATX          4  $173.98   
1     MSI B350 PC MATE        AM4         ATX          4   $85.49   
2       MSI Z370-A PRO    LGA1151         ATX          4  $109.99   
3  Gigabyte B450M DS3H        AM4   Micro ATX          4   $76.88   
4    MSI B450 TOMAHAWK        AM4         ATX          4  $110.33   

  Max RAM size Max RAM Unit  
0           64           GB  
1           64           GB  
2           64           GB  
3           64           GB  
4           64           GB

In [5]:
# Use complex regex pattern to split motherboard type into company, model and modifier columns
regex = r'(?P<Company>.*)\s(?P<Model>[A-Z]*[0-9]+(?:-?[A-Z])*)(?:\s(?P<Modifier>.*))?'
display(df['Motherboard'].str.extract(regex)
        .replace({np.nan: ''})
        .join(df)
        .drop(columns = 'Motherboard').head())

# Find all motherboards intended for gaming
display(df.assign(For_gaming = df['Motherboard'].str.contains(r'GAMING')).head())

# Replace GAMING, ULTRA, PRO, PLUS with +. Note that you must set regex flag for that
display(df.assign(Motherboard = df['Motherboard'].str.replace(r'(GAMING|ULTRA|PRO|PLUS\s+)+', '+', regex=True)).head())

Company   Model  Modifier CPU socket Form factor  RAM slots Max RAM  \
0      Asus Prime  Z370-A              LGA1151         ATX          4   64 GB   
1             MSI    B350   PC MATE        AM4         ATX          4   64 GB   
2             MSI  Z370-A       PRO    LGA1151         ATX          4   64 GB   
3  Gigabyte B450M    DS3H                  AM4   Micro ATX          4   64 GB   
4             MSI    B450  TOMAHAWK        AM4         ATX          4   64 GB   

     Price  
0  $173.98  
1   $85.49  
2  $109.99  
3   $76.88  
4  $110.33

Motherboard CPU socket Form factor  RAM slots Max RAM    Price  \
0    Asus Prime Z370-A    LGA1151         ATX          4   64 GB  $173.98   
1     MSI B350 PC MATE        AM4         ATX          4   64 GB   $85.49   
2       MSI Z370-A PRO    LGA1151         ATX          4   64 GB  $109.99   
3  Gigabyte B450M DS3H        AM4   Micro ATX          4   64 GB   $76.88   
4    MSI B450 TOMAHAWK        AM4         ATX          4   64 GB  $110.33   

   For_gaming  
0       False  
1       False  
2       False  
3       False  
4       False

Motherboard CPU socket Form factor  RAM slots Max RAM    Price
0    Asus Prime Z370-A    LGA1151         ATX          4   64 GB  $173.98
1     MSI B350 PC MATE        AM4         ATX          4   64 GB   $85.49
2         MSI Z370-A +    LGA1151         ATX          4   64 GB  $109.99
3  Gigabyte B450M DS3H        AM4   Micro ATX          4   64 GB   $76.88
4    MSI B450 TOMAHAWK        AM4         ATX          4   64 GB  $110.33

## III. What to do if an observation is described by several rows   

* Programmers like key-value tables as you can put anything into it. 
* Data analysts hate them as you can put anything into it.
* Such tables must be converted back to more wider format as follows:
   * create a multi-index defining a single observation
   * use `unstack` to push part of the multi-index to column index 
   * clean up all indices to restore status quo using `reset_index` and `columns.droplevel`

In [6]:
# Let's create a simple key-value table 
df = read_csv('pricelist.csv')
df.index.name = 'row_id'
df = df.reset_index()
df = melt(df, id_vars = 'row_id')

# Each potential purchase is defined by six rows
display(df.sort_values(['row_id', 'variable']).head(12))

# Idiomatic way to widen the table
df = df.set_index(['row_id', 'variable']).unstack().reset_index(drop=True)
df.columns = df.columns.droplevel(0)
display(df.head())       

row_id     variable              value
40        0   CPU socket            LGA1151
80        0  Form factor                ATX
160       0      Max RAM              64 GB
0         0  Motherboard  Asus Prime Z370-A
200       0        Price            $173.98
120       0    RAM slots                  4
41        1   CPU socket                AM4
81        1  Form factor                ATX
161       1      Max RAM              64 GB
1         1  Motherboard   MSI B350 PC MATE
201       1        Price             $85.49
121       1    RAM slots                  4

variable CPU socket Form factor Max RAM          Motherboard    Price  \
0           LGA1151         ATX   64 GB    Asus Prime Z370-A  $173.98   
1               AM4         ATX   64 GB     MSI B350 PC MATE   $85.49   
2           LGA1151         ATX   64 GB       MSI Z370-A PRO  $109.99   
3               AM4   Micro ATX   64 GB  Gigabyte B450M DS3H   $76.88   
4               AM4         ATX   64 GB    MSI B450 TOMAHAWK  $110.33   

variable RAM slots  
0                4  
1                4  
2                4  
3                4  
4                4

## IV. Dataframe contains duplicated field values 

* A row in a data frame can be a combination of several independent facts.
* If one of these facts is reoccuring in many rows then the data frame must be decomposed.
* To do that a data frame must be split so that all parts have the same index:
  * create unique index for joining: `df.groupby(column_list).ngroup()`  
  * select relevant columns: `df[column_list]`   
  * drop repeating rows: `drop_duplicates` 
  * add index for combining the data: `set_index`
* Index creation can be dropped if there already exists a good key column. 
* Decomposition may reveal errors and inconsistencies of the data.
* Later on the data might be joined again but then in a controlled way for particular purposes.

In [7]:
df = read_csv('billboard.csv')
display(df.head())
display(df.loc[df['song'] =='Havana', :].head())

week  position                              performer      song  \
0  2018-01-06         1                             Ed Sheeran   Perfect   
1  2018-01-06         2        Post Malone Featuring 21 Savage  Rockstar   
2  2018-01-06         3    Camila Cabello Featuring Young Thug    Havana   
3  2018-01-06         4  G-Eazy Featuring A$AP Rocky & Cardi B  No Limit   
4  2018-01-06         5                        Imagine Dragons   Thunder   

   instance previous_position  peak_position  weeks_on_chart  
0         2                 1              1              17  
1         1                 2              1              14  
2         2                 3              2              19  
3         1                 5              4              15  
4         1                 6              4              34

week  position                            performer    song  \
2    2018-01-06         3  Camila Cabello Featuring Young Thug  Havana   
101  2018-01-13         2  Camila Cabello Featuring Young Thug  Havana   
201  2018-01-20         2  Camila Cabello Featuring Young Thug  Havana   
300  2018-01-27         1  Camila Cabello Featuring Young Thug  Havana   
402  2018-02-03         3  Camila Cabello Featuring Young Thug  Havana   

     instance previous_position  peak_position  weeks_on_chart  
2           2                 3              2              19  
101         2                 3              2              20  
201         2                 2              2              21  
300         2                 2              1              22  
402         2                 1              1              23

* Values in the columns `song` and `performer` are repeated over and over.
* Values in the columns `peak_position` and `weeks_on_chart` are computable form other data.
* Computable fields are a common source of errors and inconsistencies.
* We keep them only because the data contains charts from 2018 only and we cannot recompute them.

In [8]:
# Let's add a song_id for joining the data later on 
df['song_id'] = df.groupby(['performer','song']).ngroup()

# Extract data about performers 
songs = (df[['song_id', 'performer', 'song']]
         .drop_duplicates()
         .set_index('song_id')
         .sort_values('song_id')) 
display(songs.head())

# Extract data about charts themselves
charts = (df[['week', 'position', 'song_id', 'instance', 'previous_position', 'peak_position', 'weeks_on_chart']]
         .set_index('song_id'))

display(charts.head())

# Decomposition can be reversed with join or merge methods
display(charts.join(songs).sort_values(['week', 'position']).head())
display(charts.reset_index().merge(songs.reset_index(), on = 'song_id').sort_values(['week', 'position']).head())

performer           song
song_id                                               
0        2 Chainz Featuring YG & Offset          Proud
1               2 Chainz, Drake & Quavo   Bigger > You
2                             21 Savage   Bank Account
3                   5 Seconds Of Summer  Want You Back
4                   5 Seconds Of Summer     Youngblood

week  position  instance previous_position  peak_position  \
song_id                                                                    
176      2018-01-06         1         2                 1              1   
547      2018-01-06         2         1                 2              1   
83       2018-01-06         3         2                 3              2   
214      2018-01-06         4         1                 5              4   
240      2018-01-06         5         1                 6              4   

         weeks_on_chart  
song_id                  
176                  17  
547                  14  
83                   19  
214                  15  
240                  34

week  position  instance previous_position  peak_position  \
song_id                                                                    
176      2018-01-06         1         2                 1              1   
547      2018-01-06         2         1                 2              1   
83       2018-01-06         3         2                 3              2   
214      2018-01-06         4         1                 5              4   
240      2018-01-06         5         1                 6              4   

         weeks_on_chart                              performer      song  
song_id                                                                   
176                  17                             Ed Sheeran   Perfect  
547                  14        Post Malone Featuring 21 Savage  Rockstar  
83                   19    Camila Cabello Featuring Young Thug    Havana  
214                  15  G-Eazy Featuring A$AP Rocky & Cardi B  No Limit  
240                  34                        Imagine Dragons   Thunder

song_id        week  position  instance previous_position  peak_position  \
0        176  2018-01-06         1         2                 1              1   
40       547  2018-01-06         2         1                 2              1   
67        83  2018-01-06         3         2                 3              2   
94       214  2018-01-06         4         1                 5              4   
108      240  2018-01-06         5         1                 6              4   

     weeks_on_chart                              performer      song  
0                17                             Ed Sheeran   Perfect  
40               14        Post Malone Featuring 21 Savage  Rockstar  
67               19    Camila Cabello Featuring Young Thug    Havana  
94               15  G-Eazy Featuring A$AP Rocky & Cardi B  No Limit  
108              34                        Imagine Dragons   Thunder

## V. Relevant data is scattered among several data frames

* Different data frames can be joined based on key columns or indices.
* If key columns are pushed to the index then `df.join` is appropriate:
  * data frames inside the `df.join` call must have key columns as indices
  * use `on` argument if key columns are outside of `df.index` 
  * join type is determined by `how` argument
  * arguments `lsuffix` and `rsuffix` are used to disambiguate overlapping columns
* Otherwise `df.merge` is more appropriate. It is also more flexible:
  * key columns can be named differently
  * merge type can be validated
  
* Sometimes the data is horizontally split among several data frames:
  * For instance, there is a data frame for each year with the same structure.
  
* Methods `df.append` and `pandas.concat` allow to combine such data:
  * At this phase, keep indices simple or you might get into trouble.
  * `ignore_index` – allows to merge tables with conflicting indices
  * `verify_integrity` – allows to check against duplicates in the index

In [9]:
# Let's find the top place in 2018 for each performer

# Let's use join to merge the data 
df = charts.join(songs['performer'])[['performer', 'position']]
display(df.sort_values('performer').head())

# Let's compute the top position 
display(df.groupby('performer').min()
        .rename(columns = {'position': 'best_position'})
        .sort_values('best_position', ascending = True)
        .head())

# Let's count how many weeks the performer held the best position
df = df.groupby('performer').min().rename(columns = {'position': 'best_position'})
df = df.merge(charts.join(songs), left_on = ['performer', 'best_position'], right_on = ['performer', 'position'])

# Let's compute the answer by grouping
display(df[['performer', 'best_position']]
        .groupby('performer').count()
        .rename(columns = {'best_position': 'weeks'})
        .sort_values('weeks', ascending = False)
        .head())
        
# Let's investigate how many different songs each performer has in his or her top places
df =(charts.join(songs['performer'])[['performer', 'position']]
     .groupby('performer').min()
     .rename(columns = {'position': 'best_position'}))
df = df.merge(charts.join(songs), left_on = ['performer', 'best_position'], right_on = ['performer', 'position'])
df = df[['performer', 'song']].drop_duplicates()

tmp = (df.groupby('performer').count()
       .sort_values('song', ascending = False)
       .reset_index()
       .rename(columns = {'song': 'hit_count'}))
       
display(tmp.merge(df, on = 'performer')
        .set_index('performer')
        .sort_values(['hit_count', 'performer', 'song'], ascending = False)
        .head(10))

performer  position
song_id                                          
0        2 Chainz Featuring YG & Offset        96
0        2 Chainz Featuring YG & Offset        99
1               2 Chainz, Drake & Quavo        53
1               2 Chainz, Drake & Quavo        60
2                             21 Savage        28

best_position
performer                                
Ariana Grande                           1
Travis Scott                            1
Maroon 5 Featuring Cardi B              1
Drake                                   1
XXXTENTACION                            1

weeks
performer                        
Drake                          29
Bruno Mars & Cardi B            8
Maroon 5 Featuring Cardi B      7
Ariana Grande                   6
Charlie Puth                    5

hit_count                    song
performer                                                   
Drake                              3           Nice For What
Drake                              3          In My Feelings
Drake                              3              God's Plan
Luke Combs                         2  She Got The Best Of Me
Luke Combs                         2         One Number Away
twenty one pilots                  1                Jumpsuit
lovelytheband                      1                  Broken
Zedd, Maren Morris & Grey          1              The Middle
Zedd & Elley Duhe                  1               Happy Now
Zayn Featuring Sia                 1          Dusk Till Dawn

## VI. Enforce correctness by defining classifiers

* Normally, cells in a data frame have either a numeric value or are strings.
* If a cell in a column can have finite number of potential values then it is a categorical value:
  * nominal values have no ordering such as location names
  * ordinal values have some sort of linear order such as grades
* Pandas supports categorical data for three reasons:
  * **memory compression:** list of allowed values is defined based on the data
  * **correctness checking:** first, the list of allowed values is defined, and then the data is converted to a categorical form
  * **data discretisation:** you can convert continuous values into discrete ranges with `pandas.cut`
* It is possible to define nominal and ordinal categories:
  * Ordinal category supports linear ordering of values.
* Further details can be found:
  * http://matthewrocklin.com/blog/work/2015/06/18/Categoricals
  * https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
  * https://pandas.pydata.org/docs/reference/api/pandas.CategoricalDtype.html

### Ways to define categories for correctness checking

* The safest way to define categories is through `CategoricalDtype` defined in `pandas.api.type`:
  * You define the list of potential values and whether the values are ordered or not.
  * The resulting object describes a mapping of type `code` --> `value`.

* Given a description of a category type you can convert a data frame column using `astype`:
  * As a result, unexpected values are converted to `numpy.nan` values.
  * Categorical data type is used to build an individual mapping `code` --> `value` for the column.
  * There are no references to the categorical datatype.

* The mapping can be accessed through `df['column'].cat.category` and codes `df['column'].cat.codes`:
  * By changing the labels for the codes it is possible to change the data.
  * The safest way to do it is `cat.rename_categories({code: 'label'})`.
  * **Important:** Some methods change and some do not change the original data! 
  
* **Important:** Different columns can share the same mapping `df['column'].cat.category`:
  * If you change one description the others also change!
  * To avoid this, do the type conversion with `astype` to get a new description.
  * Deep copy does not work for some reason!

In [10]:
from pandas import Categorical
from pandas.api.types import CategoricalDtype

# First define a list of potential gender values
GenderType1 = CategoricalDtype(['M', 'F'], ordered = False)
GenderType2 = CategoricalDtype(['Male', 'Female'], ordered = False)
display(GenderType1, GenderType2)

# Take a data frame and convert a column to corresponding categorical type 
df = (DataFrame({'gender': ['Male', 'Female', 'Bob', ''], 'height': [178, 153, 202, 167]})
    .assign(gender = lambda df: df['gender'].astype(GenderType2)))
display(df)

# Rename category labels by changing the labels of the category. 
# This returns a new column but keeps the original column unaltered
display(df['gender'].cat.rename_categories({'Male': 'M', 'Female':'F'}))
display(df['gender'])


# Another way for lable changes is to assign categories directly
# This will alter the original data. So do not use it! 
df['gender'].cat.categories = GenderType1.categories
display(df)

# A safe way to achieve the same effect is
display(df['gender'].cat.rename_categories(GenderType1.categories))


# Note that categories of different columns are linked 
# Changes in labelling can lead to unpredictable results
df['sex'] = df['gender']
# Change propagates to both columns
df['gender'].cat.categories = GenderType1.categories
display(df)
# Change does not propagate back for some reason
df['gender'].cat.categories = GenderType2.categories
display(df)

# Safe ways for assigning
df['sex'] = df['gender'].astype(GenderType2)
df['gender'].cat.categories = GenderType1.categories
display(df)

# Deep copy does not decouple category descriptions
df['sex'] = df['gender'].copy(deep = True)
df['gender'].cat.categories = GenderType1.categories
display(df)

CategoricalDtype(categories=['M', 'F'], ordered=False)

CategoricalDtype(categories=['Male', 'Female'], ordered=False)

gender  height
0    Male     178
1  Female     153
2     NaN     202
3     NaN     167

0      M
1      F
2    NaN
3    NaN
Name: gender, dtype: category
Categories (2, object): ['M', 'F']

0      Male
1    Female
2       NaN
3       NaN
Name: gender, dtype: category
Categories (2, object): ['Male', 'Female']

...

0      M
1      F
2    NaN
3    NaN
Name: gender, dtype: category
Categories (2, object): ['M', 'F']

gender  height  sex
0      M     178    M
1      F     153    F
2    NaN     202  NaN
3    NaN     167  NaN

gender  height  sex
0    Male     178    M
1  Female     153    F
2     NaN     202  NaN
3     NaN     167  NaN

gender  height     sex
0      M     178    Male
1      F     153  Female
2    NaN     202     NaN
3    NaN     167     NaN

gender  height  sex
0      M     178    M
1      F     153    F
2    NaN     202  NaN
3    NaN     167  NaN

In [11]:
# Categorical data types can be used for reading csv files. 
# Prescribing categorical data types can lead to missing values 
CPUSocketType = CategoricalDtype(['LGA1151', 'AM3+', 'AM4'], ordered = False)
df = read_csv('pricelist.csv', dtype = {'CPU socket': CPUSocketType})
display(df['CPU socket'].head())

# Check for missing values
display(any(df['CPU socket'].isna()))

0    LGA1151
1        AM4
2    LGA1151
3        AM4
4        AM4
Name: CPU socket, dtype: category
Categories (3, object): ['LGA1151', 'AM3+', 'AM4']

False

### Simple lossless way for reducing memory footprint

* Use `astype('category')` to convert data to categorical data wihtout loosing any values.
* The list of possible values is created based on observed values.

In [12]:
df = (songs.assign(performer = songs['performer'].astype('category'))
      .assign(song = songs['song'].astype('category'))) 
display(df.head(), df['performer'].head())  

performer           song
song_id                                               
0        2 Chainz Featuring YG & Offset          Proud
1               2 Chainz, Drake & Quavo   Bigger > You
2                             21 Savage   Bank Account
3                   5 Seconds Of Summer  Want You Back
4                   5 Seconds Of Summer     Youngblood

song_id
0    2 Chainz Featuring YG & Offset
1           2 Chainz, Drake & Quavo
2                         21 Savage
3               5 Seconds Of Summer
4               5 Seconds Of Summer
Name: performer, dtype: category
Categories (446, object): ['2 Chainz Featuring YG & Offset', '2 Chainz, Drake & Quavo', '21 Savage', '5 Seconds Of Summer', ..., 'Zedd & Elley Duhe', 'Zedd, Maren Morris & Grey', 'lovelytheband', 'twenty one pilots']

### What to do if the original data is already encoded

* Sometimes the data is already in encoded format:
  * There is a data frame of `code` --> `label` values and original data is described in codes.
  * If codes range from `0` to `n` then `Categorical.from_codes` provides a fast way to create `categorical` 
* However, this works only if codes are in the right range. 
* Otherwise you should convert `label` column to a categorical and use merge to convert original values to categorical

### Discretisation of continuous values

* If you use `pandas.cut` for data discretisation you get categorical output:
  * `bins` – specifies ranges corresponding to discrete labels
  * `labels` – gives names to data ranges
  * other options control the cutting process

In [13]:
df = (DataFrame({'gender': ['Male', 'Female', 'Bob', ''], 'height': [178, 153, 202, 167]})
    .assign(gender = lambda df: df['gender'].astype(GenderType2))
    .assign(dheight = lambda df: pd.cut(df['height'], bins = 3, labels = ['short', 'normal', 'tall'])))

display(df)

gender  height dheight
0    Male     178  normal
1  Female     153   short
2     NaN     202    tall
3     NaN     167   short

### Various operations with categories

* Information about categories can be accessed with `.cat` property:
  * `categories` – list of potential values
  * `as_ordered()` – define order between potential values
  * `as_unordered()` – define order between potential values
  * `add_categories()` – add new potential values
  * `remove_categories()` – remove potential values
  * `remove_unused_categories()` – remove unused categories
  * `rename_categories()` – relabel categories
  * `codes` – internal codes corresponding to cell values
* Categorical data can be used as ordinary data with some additional constraints:
  * Merge and concatenation can be done over the same categorical datatype.
  * It is possible to unite categoricals with `union_categoricals` from `pandas.api.types`.

In [14]:
# First define a list of potential gender values
GenderType1 = CategoricalDtype(['M', 'F'], ordered = False)
GenderType2 = CategoricalDtype(['M', 'F', 'T'], ordered = False)

df1 = (DataFrame({'gender': ['M', 'F', 'F', 'T'], 'height': [178, 153, 202, 167]})
    .assign(gender = lambda df: df['gender'].astype(GenderType1)))

df2 = (DataFrame({'gender': ['M', 'F', 'T'], 'height': [188, 152, 160]})
    .assign(gender = lambda df: df['gender'].astype(GenderType2))) 

display(df1['gender'])
display(df2['gender'])

# As categories do not match values are converted back to strings  
df3 = concat([df1, df2], ignore_index = True)
display(df3['gender'])

# Same does not ahppen if the categories match
df3 = concat([df1, df1], ignore_index = True)
display(df3['gender'])

# Let see the encoding of categorical values. See that missing value is encoded as -1
display(df1['gender'].cat.codes)

0      M
1      F
2      F
3    NaN
Name: gender, dtype: category
Categories (2, object): ['M', 'F']

0    M
1    F
2    T
Name: gender, dtype: category
Categories (3, object): ['M', 'F', 'T']

0      M
1      F
2      F
3    NaN
4      M
5      F
6      T
Name: gender, dtype: object

0      M
1      F
2      F
3    NaN
4      M
5      F
6      F
7    NaN
Name: gender, dtype: category
Categories (2, object): ['M', 'F']

0    0
1    1
2    1
3   -1
dtype: int8